In [21]:
import os
from dotenv import load_dotenv

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [22]:
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader, SeleniumURLLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.prompts import PromptTemplate

In [23]:
# website = "https://hackernoon.com/vector-databases-getting-started-with-chromadb-and-more"

# def load_document(loader_class, website_url):
#    loader = loader_class([website_url])
#    return loader.load()

# wb_loader_doc = load_document(WebBaseLoader, website)
# wb_loader_doc[0].page_content


In [24]:
# selenium_loader_doc = load_document(SeleniumURLLoader, website)
# selenium_loader_doc[0].page_content

In [25]:
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=200)
# splits = text_splitter.split_documents(selenium_loader_doc)
# splits[0]

In [26]:
import chromadb

client = chromadb.HttpClient(host="localhost", port=8000)
print("HEARTBEAT:", client.heartbeat())

HEARTBEAT: 1711205998020018000


In [27]:
import chromadb.utils.embedding_functions as embedding_functions

openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=os.getenv("OPENAI_API_KEY"), model_name="text-embedding-3-small"
)

In [28]:
documents = [
    "PA is working at SAP",
    "PA also participates in a non-profit project",
    "PA's fav pokemon is Snorlax",
    "PA likes tacos",
]
document_ids = list(map(lambda tup: f"id{tup[0]}", enumerate(documents)))

In [29]:
collection = client.get_or_create_collection(name="test_collection", embedding_function=openai_ef)

In [30]:
# collection.add(documents=documents, ids=document_ids)

In [31]:
collection.count()

4

In [32]:
# Note: the following code does not work because the embeddings cannot be retrieved

# vectordb = Chroma(persist_directory="../src/chromadb/chroma_data", collection_name="test_collection", embedding_function=openai_ef)
# vectordb._collection.count()


In [33]:
# prompt = PromptTemplate.from_template(
#     """
#     Answer the question based only on the following context:
#     Context: {context}
#     Question: {question}
#     """
# )

In [34]:
# llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [35]:
# vectorstore = Chroma.from_texts(["PA is working at SAP",
#                                 "PA also participates in a non-profit project",
#                                 "PA's fav pokemon is Snorlax",
#                                 "PA likes tacos"], embedding=OpenAIEmbeddings())
# vectorstore.persist()
# retriever = vectordb.as_retriever()

In [36]:
# rag_chain = (
#     {"context": retriever, "question": RunnablePassthrough()}
#     | prompt
#     | llm
#     | StrOutputParser()
# )

In [37]:
# response = rag_chain.invoke("What can you tell me about PA?")
# response